## **NASA Project**

### **Data Pagination, Data Cleaning, & Data Processing**


In [ ]:
import requests
from datetime import datetime, timedelta

API_KEY = "KhRzbk3Ltq3liX0BNL4Saxo5190LqJ0aseyS60mQ"  # NASA API key
BASE_URL = "https://api.nasa.gov/neo/rest/v1/feed"
START_DATE = datetime.strptime("2024-01-01", "%Y-%m-%d")

asteroid_data = []
total_asteroids = 0
record_limit = 10000
date_range = timedelta(days=7)
current_date = START_DATE

while total_asteroids < record_limit:
    # Construct start and end dates
    start_str = current_date.strftime("%Y-%m-%d")
    end_date = current_date + date_range
    end_str = end_date.strftime("%Y-%m-%d")

    # Construct the API URL with date range
    url = f"{BASE_URL}?start_date={start_str}&end_date={end_str}&api_key={API_KEY}"

    print(f"Fetching: {start_str} to {end_str}")
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        break

    data = response.json()

    # Loop through each date's NEOs
    for date, neos in data.get("near_earth_objects", {}).items():
        for neo in neos:
            # Safely get close_approach_data, as not all asteroids might have it
            close_approach_data = neo.get('close_approach_data')

            # Initialize values to None or default if close_approach_data is missing
            approach_date = None
            relative_velocity_kmph = None
            astronomical_distance = None
            miss_distance_km = None
            miss_distance_lunar = None
            orbiting_body = None

            if close_approach_data and len(close_approach_data) > 0:
                first_approach = close_approach_data[0]
                approach_date = first_approach.get('close_approach_date')
                relative_velocity_kmph = first_approach.get('relative_velocity', {}).get('kilometers_per_hour')
                astronomical_distance = first_approach.get('miss_distance', {}).get('astronomical')
                miss_distance_km = first_approach.get('miss_distance', {}).get('kilometers')
                miss_distance_lunar = first_approach.get('miss_distance', {}).get('lunar')
                orbiting_body = first_approach.get('orbiting_body')

            asteroid = {
                'id': int(neo['id']),
                'neo_reference_id': neo['neo_reference_id'],
                'name':neo['name'],
                'absolute_magnitude_h': neo['absolute_magnitude_h'],
                'estimated_diameter_min_km': neo['estimated_diameter']['kilometers']['estimated_diameter_min'],
                'estimated_diameter_max_km': neo['estimated_diameter']['kilometers']['estimated_diameter_max'],
                'is_potentially_hazardous_asteroid': neo['is_potentially_hazardous_asteroid'],
                'close_approach_date': approach_date,
                'relative_velocity_kmph': float(relative_velocity_kmph) if relative_velocity_kmph else None,
                'astronomical': float(astronomical_distance) if astronomical_distance else None,
                'miss_distance_km': float(miss_distance_km) if miss_distance_km else None,
                'miss_distance_lunar': float(miss_distance_lunar) if miss_distance_lunar else None,
                'orbiting_body': orbiting_body
            }
            asteroid_data.append(asteroid)
            total_asteroids += 1

            if total_asteroids >= record_limit:
                break
        if total_asteroids >= record_limit:
            break

    # Update the date for the next 7-day range
    current_date += date_range
print("\n🛰️ First 5 Asteroid Records:")
for i, asteroid in enumerate(asteroid_data[:5], start=0):
    print(f"\nAsteroid #{i}")
    for key, value in asteroid.items():
        print(f"{key}: {value}")

print(f"\n✅ Total Asteroids Collected: {len(asteroid_data)}")

# SQL Insertion

!pip install pandas
!pip install mysql-connector-python
!pip install streamlit

import mysql.connector
API_KEY = "KhRzbk3Ltq3liX0BNL4Saxo5190LqJ0aseyS60mQ"
START_DATE = datetime(2024, 1, 1)


conn = mysql.connector.connect(host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                               user = "2tX8YVbJJrde2n4.root",
                               password = "YtafUUwGk86fMYaU",
                               port=4000)
conn.cursor()
cursor = conn.cursor()


conn = mysql.connector.connect(host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                               user = "2tX8YVbJJrde2n4.root",
                               password = "YtafUUwGk86fMYaU",
                               port=4000,
                               database = "NASA")
cursor = conn.cursor()


# Drop tables if they exist to ensure a clean creation
drop_close_approach_table = "DROP TABLE IF EXISTS close_approach;"
drop_asteroids_table = "DROP TABLE IF EXISTS asteroids;"
cursor.execute(drop_close_approach_table)
cursor.execute(drop_asteroids_table)

# Create 'asteroids' table
create_asteroids_table = """
CREATE TABLE IF NOT EXISTS asteroids (
    id BIGINT UNSIGNED PRIMARY KEY,
    neo_reference_id VARCHAR(20),
    name VARCHAR(255),
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN);"""

# Create 'close_approach' table
create_close_approach_table = """
CREATE TABLE IF NOT EXISTS close_approach (
    approach_id INT AUTO_INCREMENT PRIMARY KEY,
    asteroid_id BIGINT UNSIGNED,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR(50),
    FOREIGN KEY (asteroid_id) REFERENCES asteroids(id));"""
cursor.execute(create_asteroids_table)
cursor.execute(create_close_approach_table)
#cursor.execute("SHOW TABLES")
#for table in cursor:
  #  print(table)

conn.commit()

import pandas as pd
df = pd.DataFrame(asteroid_data)
df


insert = "INSERT IGNORE into asteroids values(%s, %s, %s, %s, %s, %s, %s)"

for asteroid in asteroid_data:
 values = (asteroid['id'],
           asteroid['neo_reference_id'],
           asteroid['name'],
           asteroid['absolute_magnitude_h'],
           asteroid['estimated_diameter_min_km'],
           asteroid['estimated_diameter_max_km'],
           asteroid['is_potentially_hazardous_asteroid'])
 cursor.execute(insert,values)
conn.commit()

insert = "INSERT into close_approach (asteroid_id, close_approach_date, relative_velocity_kmph, astronomical, miss_distance_km, miss_distance_lunar, orbiting_body) values(%s, %s, %s, %s, %s, %s, %s)"

for asteroid in asteroid_data:
    values = (asteroid['id'],
              asteroid['close_approach_date'],
              asteroid['relative_velocity_kmph'],
              asteroid['astronomical'],
              asteroid['miss_distance_km'],
              asteroid['miss_distance_lunar'],
              asteroid['orbiting_body'])
    cursor.execute(insert,values)
conn.commit()

cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [asteroid[0] for asteroid in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted

cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [i[0] for i in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns = columns)
new_df # shows all the values inserted

In [ ]:
conn = mysql.connector.connect(host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
                               user = "2tX8YVbJJrde2n4.root",
                               password = "YtafUUwGk86fMYaU",
                               port=4000,
                               database = "NASA")
cursor = conn.cursor()


In [ ]:
cursor.execute("SELECT * FROM asteroids")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [asteroid[0] for asteroid in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns=columns)
new_df # shows all the values inserted


,id,neo_reference_id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid
0,2000887,2000887,887 Alinda (A918 AA),13.82,4.576726,10.233872,0
1,2001685,2001685,1685 Toro (1948 OA),14.28,3.703011,8.280184,0
2,2002063,2002063,2063 Bacchus (1977 HB),17.21,0.960627,2.148028,0
3,2002340,2002340,2340 Hathor (1976 UA),20.43,0.218049,0.487573,1
4,2004034,2004034,4034 Vishnu (1986 PA),18.49,0.532789,1.191352,1
...,...,...,...,...,...,...,...
7496,54533502,54533502,(2025 KR4),26.01,0.016694,0.037328,0
7497,54533907,54533907,(2025 KZ8),25.03,0.026215,0.058619,0
7498,54536909,54536909,(2025 MU34),24.41,0.034878,0.077990,0
7499,54537502,54537502,(2025 MG89),23.86,0.044932,0.100471,0


In [ ]:
cursor.execute("SELECT * FROM close_approach")
data = cursor.fetchall()
#fetches the names of your columns in dataframe(cursor.description holds the columns info)
columns = [asteroid[0] for asteroid in cursor.description]
import pandas as pd
# using the result fetched in cursor, create a new dataframe
new_df = pd.DataFrame(data,columns = columns)
new_df # shows all the values inserted

,approach_id,asteroid_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body
0,1,2415949,2024-01-02,57205.895,0.337254,50452410.0,131.191620,Earth
1,2,3160747,2024-01-02,38589.055,0.132343,19798170.0,51.481270,Earth
2,3,3309828,2024-01-02,56413.016,0.167013,24984732.0,64.967910,Earth
3,4,3457842,2024-01-02,21891.120,0.492051,73609800.0,191.407880,Earth
4,5,3553062,2024-01-02,31468.979,0.235802,35275516.0,91.727070,Earth
...,...,...,...,...,...,...,...,...
9995,9996,54282890,2025-04-24,32068.238,0.172322,25779030.0,67.033325,Earth
9996,9997,54338709,2025-04-24,132519.600,0.242696,36306756.0,94.408615,Earth
9997,9998,54372216,2025-04-24,55509.160,0.351616,52601052.0,136.778750,Earth
9998,9999,54397132,2025-04-24,52793.000,0.204966,30662420.0,79.731620,Earth


# **Streamlit**

In [1]:
!pip install mysql-connector-python
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 66.2 MB/s eta 0:00:00


In [15]:
%%writefile NASA.py


#!pip install mysql-connector-python
#!pip install pandas
import streamlit as st
import mysql.connector
import pandas as pd
from datetime import date

# ---------- Database Connection ----------
def get_connection():
    return mysql.connector.connect(
        host="gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
        user="2tX8YVbJJrde2n4.root",
        password="YtafUUwGk86fMYaU",
        database="NASA"
    )
# ---------- Predefined Queries ----------
QUERIES = {
    "Count how many times each astroid has approached Earth":"""
    SELECT
    asteroid_id,COUNT(*) AS approach_count
    FROM close_approach
    GROUP BY
    asteroid_id
    ORDER BY
    approach_count DESC""",

    "Average velocity of each asteroid over multiple approaches":"""
    SELECT
    asteroid_id,
    AVG(relative_velocity_kmph) AS average_velocity_kmph
    FROM
    close_approach
    GROUP BY
    asteroid_id
    ORDER BY
    average_velocity_kmph DESC""",

    "List top 10 fastest asteroids":"""
    SELECT
    a.name,
    ca.relative_velocity_kmph
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    ORDER BY
    ca.relative_velocity_kmph DESC
    LIMIT 10""",

    "Find potentially hazardous asteroids that have approached Earth more than 3 times":"""
    SELECT
    a.name,
    COUNT(*) AS approach_count
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    WHERE
    a.is_potentially_hazardous_asteroid = TRUE
    GROUP BY
    a.name
    HAVING
    COUNT(*) > 3
    ORDER BY
    approach_count DESC""",

    "Find the month with the most asteroid approaches":"""
    SELECT
    DATE_FORMAT(close_approach_date, '%Y-%m') AS approach_month,
    COUNT(*) AS approach_count
    FROM
    close_approach
    GROUP BY
    approach_month
    ORDER BY
    approach_count DESC
    LIMIT 1""",

    "Get the asteroid with the fastest ever approach speed":"""
    SELECT
    a.name,
    ca.relative_velocity_kmph
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    ORDER BY
    ca.relative_velocity_kmph DESC
    LIMIT 1""",

    "Sort asteroids by maximum estimated diameter (descending)":"""
    SELECT
    name,
    estimated_diameter_max_km
    FROM
    asteroids
    ORDER BY
    estimated_diameter_max_km DESC""",

    "An asteroid whose closest approach is getting nearer over time(Hint: Use ORDER BY close_approach_date and look at miss_distance)":"""
    SELECT
    a.name,
    ca.close_approach_date,
    ca.miss_distance_km
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    WHERE
    a.id IN (SELECT asteroid_id FROM close_approach GROUP BY asteroid_id HAVING COUNT(*) > 1)
    ORDER BY a.id, ca.close_approach_date""",

    "Display the name of each asteroid along with the date and miss distance of its closest approach to Earth":"""
    SELECT
    a.name,
    ca.close_approach_date,
    ca.miss_distance_km
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id""",

    "List names of asteroids that approached Earth with velocity > 50,000 km/h":"""
    SELECT
    a.name
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    WHERE
    ca.relative_velocity_kmph > 50000""",

    "Count how many approaches happened per month":"""
    SELECT
    DATE_FORMAT(close_approach_date, '%Y-%m') AS approach_month,
    COUNT(*) AS approach_count
    FROM
    close_approach
    GROUP BY
    approach_month
    ORDER BY
    approach_month""",

    "Find asteroid with the highest brightness (lowest magnitude value)":"""
    SELECT
    name,
    absolute_magnitude_h
    FROM
    asteroids
    ORDER BY
    absolute_magnitude_h ASC
    LIMIT 1""",

    "Get number of hazardous vs non-hazardous asteroids":"""
    SELECT
    is_potentially_hazardous_asteroid,
    COUNT(*) AS count
    FROM
    asteroids
    GROUP BY
    is_potentially_hazardous_asteroid""",

    "Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance":"""
    SELECT
    a.name,
    ca.close_approach_date,
    ca.miss_distance_lunar
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    WHERE
    ca.miss_distance_lunar < 1""",

    "Find asteroids that came within 0.05 AU(astronomical distance)":"""
    SELECT
    a.name,
    ca.close_approach_date,
    ca.astronomical
    FROM
    asteroids a
    JOIN
    close_approach ca ON a.id = ca.asteroid_id
    WHERE
    ca.astronomical < 0.05""",

    "Top 10 Largest Asteroids": """
     SELECT name, estimated_diameter_max_km
     FROM asteroids
     ORDER BY estimated_diameter_max_km DESC
     LIMIT 10""",

    "Hazardous Asteroids Only": """
     SELECT name, is_potentially_hazardous_asteroid
     FROM asteroids
     WHERE is_potentially_hazardous_asteroid = 1""",

    "Find Non-Hazardous astroids only":"""
    SELECT
    name,
    is_potentially_hazardous_asteroid
    FROM
    asteroids
    WHERE
    is_potentially_hazardous_asteroid = FALSE""",

    "Top 10 smallest Asteroids":"""
    SELECT
    name,
    estimated_diameter_min_km
    FROM
    asteroids
    ORDER BY
    estimated_diameter_min_km ASC
    LIMIT 10;""",

    "Countdown the asteroids that came 0.05 AU to 0.10 AU(astronomical distance)":"""
    SELECT
    COUNT(*) AS asteroid_count
    FROM
    close_approach
    WHERE
    astronomical BETWEEN 0.05 AND 0.10;"""
}


# ---------- Sidebar ----------
st.sidebar.title("Asteroid Approaches")
st.sidebar.title("Filter criteria")
query_choice = st.sidebar.selectbox("Select  Query", list(QUERIES.keys()))

# ---------- Database Query Execution ----------
def run_query(query):
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    return pd.DataFrame(result)

#-----------Header section----------------
st.title("🚀NASA ASTEROID TRACKER⭐")   # Largest and boldest

# ---------- Filters Section ----------

#st.sidebar.title("Filter criteria")

# Row 1 — Date, AU, Lunar Distance
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("**Astronomical Units (AU)**")
    au = st.slider("AU Range", 0.0, 1.0, (0.1, 0.5))

with col2:
    st.markdown("**Lunar Distances (LD)**")
    ld = st.slider("LD Range", 0.0, 100.0, (10.0, 50.0))

with col3:
    st.markdown("**Close Approach Date**")
    date_range = st.date_input("Close Approach Date", [date(2024, 1, 1), date(2024, 12, 31)])
    start_date = date_range[0]
    end_date = date_range[1]
st.write(f"Filtering from {start_date} to {end_date}")
   # close_date = st.date_input("Select Date Range", [date(2024, 1, 1), date(2024, 12, 31)])

# Row 2 — Velocity, Diameter, Hazardous
col4, col5, col6 = st.columns(3)

with col4:
    st.markdown("**Velocity (km/h)**")
    velocity = st.slider("Velocity", 0.0, 100000.0, (10000.0, 50000.0))

with col5:
    st.markdown("**Estimated Diameter (km)**")
    diameter = st.slider("Diameter", 0.0, 10.0, (0.1, 5.0))

with col6:
    st.markdown("**Hazardous Asteroid**")
    hazardous = st.selectbox("Is Hazardous?", ["All", "Yes", "No"])

    # ---------- Display Query Result ----------
if query_choice:
    st.subheader(query_choice)
    df_result = run_query(QUERIES[query_choice])

    st.dataframe(df_result)


# ---------- Custom Query ----------
st.subheader("Filtered Asteroid Data")

def build_custom_query():
    base_query = """
        SELECT a.name, ca.close_approach_date, ca.relative_velocity_kmph,
               ca.miss_distance_lunar, ca.astronomical,
               a.estimated_diameter_min_km, a.estimated_diameter_max_km,
               a.is_potentially_hazardous_asteroid
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.asteroid_id
        WHERE ca.close_approach_date BETWEEN %s AND %s
        AND ca.astronomical BETWEEN %s AND %s
        AND ca.miss_distance_lunar BETWEEN %s AND %s
        AND ca.relative_velocity_kmph BETWEEN %s AND %s
        AND a.estimated_diameter_max_km BETWEEN %s AND %s
    """
    params = [
        date_range[0], date_range[1],
        au[0], au[1],
        ld[0], ld[1],
        velocity[0], velocity[1],
        diameter[0], diameter[1]
    ]

    if hazardous == "Yes":
        base_query += " AND a.is_potentially_hazardous_asteroid = 1"
    elif hazardous == "No":
        base_query += " AND a.is_potentially_hazardous_asteroid = 0"

    return base_query, params

custom_query, custom_params = build_custom_query()


# Run and display custom filtered query
try:
    conn = get_connection()
    cursor = conn.cursor(dictionary=True)
    cursor.execute(custom_query, custom_params)
    filtered_data = pd.DataFrame(cursor.fetchall())
    st.dataframe(filtered_data)
    conn.close()
except Exception as e:
    st.error(f"Error executing query: {e}")

Overwriting NASA.py


In [13]:
!pip install streamlit streamlit_option_menu

In [5]:
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you(copy that)

34.80.255.41


In [16]:
! streamlit run NASA.py & npx localtunnel --port 8501  # replace the file name accordingly

⠙

⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.255.41:8501

⠼⠴⠦⠧your url is: https://seven-cups-stay.loca.lt
  Stopping...
^C
